# Lab 1: 前提条件とインフラストラクチャのセットアップ

## 概要
ワークショップのすべての前提条件を確認し、AWS 上に CRM アプリケーションスタック（EC2 + NGINX + DynamoDB）をデプロイします。

## 目標
**パート 1: 前提条件**
- Python バージョン（3.10 以上）の確認
- AWS アカウントと認証情報の確認
- ワークショップの依存関係のインストール
- Bedrock AgentCore SDK とスターターツールキットの確認
- Bedrock モデルアクセスのテスト
- 共有コンテキスト用の Agent Memory のセットアップ
- Amazon Cognito を使用したユーザーおよびエージェント ID のセットアップ

**パート 2: インフラストラクチャのセットアップ**
- AWS インフラストラクチャのプロビジョニング: EC2 インスタンス、NGINX、DynamoDB、CloudWatch
- サンプル CRM アプリケーションのデプロイ
- 障害をシミュレートするフォールトインジェクションスクリプトの作成
- CloudWatch モニタリングのセットアップ
- インフラストラクチャが実行中でアクセス可能であることの確認

## 学習内容
- ワークショップの前提条件とセットアップワークフロー
- インシデント対応テスト用のフォールトインジェクションパターン
- 診断用の CloudWatch ログとメトリクスのセットアップ

## 1. Python バージョンの確認

In [ ]:
import sys
print(f"Python version: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
assert sys.version_info >= (3, 10), "Python 3.10+ required"
print("✅ Python version check passed")

## 2. ワークショップの依存関係のインストール

In [ ]:
%pip install -q -r requirements.txt
print("✅ Workshop dependencies installed")

## 3. AWS 設定の確認

In [ ]:
import boto3
from lab_helpers.config import AWS_REGION, AWS_PROFILE, MODEL_ID, WORKSHOP_NAME
from lab_helpers.lab_01.infrastructure import get_app_url

# Display configuration
print(f"Workshop Name: {WORKSHOP_NAME}")
print(f"AWS Region: {AWS_REGION}")
print(f"Model ID: {MODEL_ID}\n")

# Verify AWS credentials
session = boto3.Session(profile_name=AWS_PROFILE, region_name=AWS_REGION)
sts = session.client('sts')
identity = sts.get_caller_identity()

print(f"✅ AWS Account: {identity['Account']}")
print(f"✅ AWS User/Role: {identity['Arn']}")

## 4. Bedrock モデルアクセスのテスト

In [ ]:
import boto3
from lab_helpers.config import AWS_REGION, MODEL_ID, AWS_PROFILE

session = boto3.Session(profile_name=AWS_PROFILE, region_name=AWS_REGION)
bedrock = session.client('bedrock', region_name=AWS_REGION)

# Verify model access
try:
    model = bedrock.get_foundation_model(modelIdentifier=MODEL_ID)
    print(f"Model ID: {MODEL_ID}")
    print(f"✅ Bedrock model access verified")
except Exception as e:
    print(f"❌ Error accessing model: {e}")
    raise

## 5. AgentCore コンポーネントの確認

In [ ]:
import importlib

packages = ['bedrock_agentcore', 'strands', 'boto3', 'pydantic']

for package in packages:
    try:
        mod = importlib.import_module(package)
        version = getattr(mod, '__version__', 'installed')
        print(f"✅ {package:<20} {version}")
    except ImportError:
        print(f"❌ {package:<20} NOT FOUND")

print("\n✅ All core packages verified")

## サマリー
✅ すべての前提条件が確認されました。Lab 1: インフラストラクチャのセットアップとフォールトインジェクションに進む準備ができました。

## パート 1.5: Cognito のセットアップ（Lab 3-5 の認証）

### 概要

このセクションでは、Lab 3-5 で使用する認証インフラストラクチャ用に AWS Cognito をセットアップします：

**作成するもの:**
- Cognito ユーザープール: `aiml301-UserPool`
- **2つのユーザーグループ**（新規）:
  - **developers**: 修復計画を作成するユーザー
  - **approvers**: 計画を承認して実行するユーザー
- **2つのアプリクライアント**:
  - **ユーザー認証クライアント**（パブリック）: OAuth サポートを使用したエンドユーザー認証用
  - **M2M クライアント**（コンフィデンシャル）: Gateway から Runtime へのサービス間認証用
- **リソースサーバー**: きめ細かい認可用のカスタムスコープ（`mcp.invoke`、`runtime.access`）
- **ユーザープールドメイン**: OAuth2 トークンエンドポイント
- **2人のテストユーザー**:
  - **開発者ユーザー**: `testuser@aiml301.example.com`（`developers` グループのメンバー）
  - **承認者ユーザー**: `approver@aiml301.example.com`（`approvers` グループのメンバー）

**認証フロー:**
1. **ユーザー認証**（クライアント → Gateway）: エンドユーザーが認証情報で認証し、グループメンバーシップを含む JWT トークンを受け取る
2. **M2M 認証**（Gateway → Runtime）: Gateway がクライアント認証情報グラントを使用して Runtime アクセス用の M2M トークンを取得

**マルチアクターワークフロー（Lab-03）:**
- **開発者**がログイン → 修復計画を作成 → ブロックされる（承認が必要）
- **承認者**がログイン → 保留中のインシデントを発見 → 計画をレビュー → 実行を承認
- **開発者**が戻る → 共有メモリで承認を確認 → 承認されたステップを実行

**JWT トークンクレーム:**
セットアップ後、JWT ID トークンには以下が含まれます：
```json
{
  "email": "developer@aiml301.example.com",
  "cognito:username": "developer@aiml301.example.com",
  "cognito:groups": ["developers"],
  "sub": "uuid",
  "scope": "openid profile email custom-scopes"
}
```

**なぜグループを使用するのか？**
- **ロールベースの認可**: Gateway は実行を許可する前にユーザーが `approvers` グループに属しているかどうかを確認できる
- **インシデントルーティング**: 保留中のインシデントについては承認者にのみ通知
- **監査証跡**: メモリレコードには各アクションを実行したロールが記録される
- **アクター識別**: `email` クレームは UUID の代わりに読みやすい actor_id を提供

### 目標

✅ 一元化された認証のための Cognito インフラストラクチャのセットアップ
✅ ロールベースのアクセス制御用のユーザーグループの作成
✅ デュアル認証モードの有効化: ユーザーベースとサービス間
✅ きめ細かい認可スコープの作成
✅ リッチな JWT ID トークン用の OAuth フローの有効化
✅ 後続のラボで使用するための SSM Parameter Store への設定の保存

#### 1. Cognito セットアップの実行

In [ ]:
from lab_helpers.cognito_setup import setup_cognito_complete

# Execute complete Cognito setup workflow
cognito_config = setup_cognito_complete()

print("\n" + "="*70)
print("COGNITO SETUP COMPLETE")
print("="*70)
print("Cognito User Pool ID: ", cognito_config['user_pool_id'])

## パート 1.6: Lab 2-5 用のメモリセットアップ

このセクションでは、すべてのエージェントラボ（2-5）で会話履歴とセッション管理に使用される共有 AgentCore Memory リソースを作成します。

### 作成するもの:
- 7日間の有効期限を持つ AgentCore Memory リソース
- Lab 2-5 で簡単にアクセスできるよう memory_id を Parameter Store に保存
- Lab 2-4 での静的セッション追跡用のデフォルトセッション ID を保存

### 主な学習ポイント:
Memory は、エージェント呼び出し間およびマルチターン会話でのコンテキストの永続化を可能にします。すべてのラボでこの単一のメモリリソースを共有します。

### 目標
✅ AgentCore Memory リソースの作成  
✅ メモリ設定を Parameter Store に保存  
✅ ダウンストリームエージェントの会話履歴読み込みを有効化

In [ ]:
### 1.6.1: Create AgentCore Memory Resource

from bedrock_agentcore.memory import MemoryClient
from lab_helpers.constants import PARAMETER_PATHS
from datetime import datetime

memory_client = MemoryClient(region_name=AWS_REGION)
memory_name = f"{PARAMETER_PATHS['memory']['memory_name_prefix']}_{datetime.now().strftime('%Y%m%d%H%M%S')}"

print(f"Creating memory: {memory_name}")
memory = memory_client.create_memory_and_wait(
    name=memory_name,
    description="SRE Agent Shared Short-Term Memory for Labs 2-5",
    strategies=[],
    event_expiry_days=7,
    max_wait=600,
    poll_interval=10
)

memory_id = memory['id']
print(f"✅ Memory created: {memory_id} (Status: ACTIVE, Expiry: 7 days)")

In [ ]:
### 1.6.2: Store Memory Configuration in Parameter Store

from lab_helpers.parameter_store import put_parameter

# Store memory_id for Labs 2-5
put_parameter(
    PARAMETER_PATHS['memory']['memory_id'],
    memory_id,
    description="Memory ID for agent conversation history",
    region_name=AWS_REGION
)

# Store default session ID for Labs 2-4
put_parameter(
    PARAMETER_PATHS['memory']['default_session_id'],
    "crm-session-id",
    description="Default session ID for Labs 2-4",
    region_name=AWS_REGION
)

print(f"✅ PSM Keys stored:")
print(f"   • {PARAMETER_PATHS['memory']['memory_id']} = {memory_id}")
print(f"   • {PARAMETER_PATHS['memory']['default_session_id']} = crm-session-id")

### サマリー: メモリセットアップ完了

✅ **AgentCore Memory リソースを作成**
- すべてのラボ（2-5）で単一の共有メモリリソース
- コスト管理のための自動 7 日間有効期限
- マルチターン会話とコンテキスト読み込みをサポート

✅ **Parameter Store の設定**
- Lab 2-5 で取得するための Memory ID を保存
- Lab 2-4 用のデフォルトセッション ID を利用可能
- 一元化された設定パターンに従う

**次のステップ:**
- Lab 2: memory_id を取得してメモリフックを初期化
- Lab 3-4: 修復/予防エージェントに同じメモリを使用
- Lab 5: 共有メモリを使用したマルチエージェントオーケストレーション

## パート 2: インフラストラクチャのセットアップと CRM アプリケーションのデプロイ

インフラストラクチャのセットアップと CRM アプリケーションのデプロイは、ワークショップセットアップの一部として自動化されています。

次のセクションに進んでください。

In [ ]:
# Try url with both port 80 and 8080
print(f"Click here to access the CRM App UI: '{get_app_url()}'")


## 1. フォールトインジェクションユーティリティのセットアップ

このセクションでは、インフラストラクチャ障害を注入するツールを準備し、デプロイに組み込まれた事前作成済みの障害をレビューします。ワークショップには包括的な SRE トレーニング用に**合計 4 つの障害**が含まれています：

- **障害 1: DynamoDB スロットリング** - テーブル容量を削減して ProvisionedThroughputExceededException をトリガー
- **障害 2: IAM 権限の問題** - EC2 ロールの権限を制限して AccessDenied エラーを発生

これらの障害は、ワークショップ全体を通じて、異なる障害モードと検出方法にわたる診断エージェントの診断能力をテストするために使用されます。

In [ ]:
from lab_helpers.lab_01.fault_injection import (
    initialize_fault_injection,
    inject_dynamodb_throttling,
    inject_iam_permissions,
)

# Initialize AWS clients and retrieve infrastructure resource IDs from SSM
print("Initializing fault injection utilities...")
resources = initialize_fault_injection(AWS_REGION, AWS_PROFILE)

print(f"\nDiscovered Infrastructure Resources:")
print(f"  Nginx Instance: {resources.get('nginx_instance_id', 'Not found')}")
print(f"  App Instance: {resources.get('app_instance_id', 'Not found')}")
print(f"  CRM Activities Table: {resources.get('crm_activities_table_name', 'Not found')}")
print(f"  CRM Customers Table: {resources.get('crm_customers_table_name', 'Not found')}")
print(f"  CRM Deals Table: {resources.get('crm_deals_table_name', 'Not found')}")
print(f"  EC2 Role: {resources.get('ec2_role_name', 'Not found')}")
print(f"  Public ALB DNS: {resources.get('public_alb_dns', 'Not found')}")

print("\n✅ Fault injection utilities ready")

## 2. インフラストラクチャの確認

障害を注入する前に、CloudFormation スタックが必要なすべてのリソースを作成し、それらが正常であることを確認しましょう。

In [ ]:
from lab_helpers.lab_01.infrastructure import (
    verify_ec2_instances,
    verify_dynamodb_tables,
    verify_alb_health,
    verify_cloudwatch_logs
)

print("Verifying infrastructure components...\n")

# Verify EC2 instances are running
ec2_status = verify_ec2_instances(resources, AWS_REGION, AWS_PROFILE)

# Verify DynamoDB tables exist and are accessible
dynamodb_status = verify_dynamodb_tables(resources, AWS_REGION, AWS_PROFILE)

# Verify ALB targets are healthy
alb_status = verify_alb_health(resources, AWS_REGION, AWS_PROFILE)

# Verify CloudWatch log groups exist
logs_status = verify_cloudwatch_logs(AWS_REGION, AWS_PROFILE)

if all([ec2_status, dynamodb_status, alb_status, logs_status]):
    print("\n✅ All infrastructure components verified and healthy")
else:
    print("\n⚠️  Some infrastructure components failed verification")

## 3. フォールトインジェクションのテストと事前作成済み障害のレビュー

このセクションでは、2 つのインフラストラクチャ障害を注入し、デプロイに組み込まれた 2 つの追加障害をレビューします。これら **4 つの障害**を合わせることで、診断エージェントの包括的なトレーニングシナリオを提供します。

### 障害 1: DynamoDB スロットリング

**概要:**
DynamoDB スロットリングは、アプリケーションがテーブルのプロビジョニングされた読み取り/書き込み容量を超えた場合に発生します。これは以下の場合に発生する可能性がある一般的な本番環境の問題です：
- トラフィックスパイクがプロビジョニングされた容量を予期せず超過した場合
- テーブルの容量ユニットが不十分に設定されている場合

**障害の注入方法:**
`inject_dynamodb_throttling()` ヘルパー関数は以下のようにシミュレートします：
- メトリクステーブルを `PAY_PER_REQUEST`（無制限）から `PROVISIONED` 課金モードに変換
- 極めて低い容量制限を設定: **1 読み取り容量ユニット**と **1 書き込み容量ユニット**
- これにより、テーブルは毎秒約 1 回の読み取りと 1 回の書き込み操作しか処理できなくなる
- 通常のアプリケーション負荷はこれらの制限を即座に超過する

**予想される影響:**
- アプリケーションログに `ProvisionedThroughputExceededException` エラー
- リクエストがスロットリングされてリトライされるため、レイテンシが増加
- CloudWatch メトリクスがスロットリングされたリクエストを表示

In [ ]:
# Execute DynamoDB throttling fault injection
success = inject_dynamodb_throttling(resources, AWS_REGION, AWS_PROFILE)

if success:
    print("✅ DynamoDB throttling fault injected successfully")
    print("   → Table converted to PROVISIONED mode with 1 RCU/1 WCU")
    print("   → Normal application load will now trigger throttling")
else:
    print("❌ Failed to inject DynamoDB throttling fault")

### アプリケーションテーブルへの負荷 

それでは、エンドポイントに負荷テストを実行してみましょう。30 秒間、毎秒 20 の同時リクエストを送信します。この負荷はそれほど大きくありませんが、テーブル容量のプロビジョニングの設定ミスにより、アプリは[アプリケーションログ](https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logsV2:log-groups/log-group/$252Faws$252Fsre-workshop$252Fcrm-application)に `ProvisionedThroughputExceededException` エラーを表示し、CloudWatch メトリクスがスロットリングされたリクエストを表示するはずです。負荷テスト中に Customers タブにアクセスしようとすると、データの読み込みに問題が発生します。

In [ ]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor

alb_dns = resources['public_alb_dns']
url = f"http://{alb_dns}:8080/api/customers"

def make_request(i):
    try:
        requests.get(url, timeout=5)
    except:
        pass

for second in range(1, 31):
    with ThreadPoolExecutor(max_workers=50) as executor:
        executor.map(make_request, range(50))

    if second % 10 == 0:
        print(f"Progress: {second}/30 seconds")

    time.sleep(1)

print("\n✓ Load test complete")

### 障害 2: IAM 権限の問題

**概要:**
IAM 権限の問題は、アプリケーションが AWS リソースにアクセスするための必要な権限を持っていない場合に発生します。これは以下によって引き起こされることが多い、最も一般的な本番環境の問題の 1 つです：
- テストなしで適用された過度に制限的なセキュリティポリシー
- 信頼ポリシーの変更によるロールの引き受け失敗
- セキュリティチームによる包括的な拒否ポリシーの適用

**障害の注入方法:**
ヘルパー関数 `inject_iam_permissions()` は以下のようにシミュレートします：
- アプリケーションサーバーで使用される EC2 インスタンス IAM ロールを特定
- 元の DynamoDB アクセスポリシーをバックアップ
- 主要な DynamoDB 操作に対する明示的な **Deny** ポリシーで置き換え
- 対象: `PutItem`、`GetItem`、`Query`、`Scan`、`UpdateItem`、`DeleteItem`
- Deny ポリシーは Allow ポリシーを上書きするため、データベースアクセスが即座にブロックされる

**予想される影響:**
- データベース操作に対するアプリケーションログに `AccessDenied` 例外
- DynamoDB アクセスを必要とする機能の完全な障害

In [ ]:
# Execute IAM permission fault injection
success = inject_iam_permissions(resources, AWS_REGION, AWS_PROFILE)

if success:
    print("✅ IAM permission fault injected successfully")
    print(f"   → EC2 role '{resources.get('ec2_role_name', 'Unknown')}' now has Deny policy")
    print("   → All DynamoDB operations will return AccessDenied")
else:
    print("❌ Failed to inject IAM permission fault")

アプリケーションを呼び出したときにどのような応答が得られるかテストしてみましょう。API が DynamoDB からデータを取得できないバックエンドの問題により、エラー 500 が表示されるはずです。
**注意**: IAM 権限が伝播するまで数分かかる場合があります。500 エラーが表示されない場合は、しばらく待ってから再試行してください。

In [ ]:
time.sleep(180)
alb_dns = resources['public_alb_dns']

url = f"http://{alb_dns}:8080/api/deals"

print(f"\nGenerating 5 requests to trigger IAM errors...")
print(f"Target: {url}\n")

for i in range(10):
    try:
        response = requests.get(url, timeout=5)
        print(f"Request {i+1} - Status: {response.status_code}")
    except Exception as e:
        print(f"Request {i+1} - Error: {str(e)}")

    time.sleep(1)  # Small delay to avoid overwhelming

print("\n✓ Load complete - waiting 10 seconds for logs to propagate...")
time.sleep(10)


## サマリー

✅ 前提条件が確認され、インフラストラクチャがデプロイされました。CRM アプリケーションが実行中で、CloudWatch を通じて監視されています。エージェントがトラブルシューティングするための実際の本番環境の問題をシミュレートする障害を注入しました。

次へ: Lab 2 - 診断エージェントの構築 (Lab-02-diagnostics-agent.ipynb)

In [ ]:
# Try url with both port 80 and 8080
print(f"Click here to access the CRM App UI: '{get_app_url()}'")
